In [ ]:
import numpy as np
%matplotlib notebook
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, RateRule, \
                      AssignmentRule, FunctionDefinition
from gillespy2 import SSACSolver
# from gillespy2 import TauLeapingCSolver
# from gillespy2 import ODECSolver
# from gillespy2 import TauHybridSolver

# Vilar_Oscillator

In [ ]:
class VilarOscillator(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Vilar_Oscillator")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="alpha_a", expression="50"))
        self.add_parameter(Parameter(name="alpha_a_prime", expression="500"))
        self.add_parameter(Parameter(name="alpha_r", expression="0.01"))
        self.add_parameter(Parameter(name="alpha_r_prime", expression="50"))
        self.add_parameter(Parameter(name="beta_a", expression="50"))
        self.add_parameter(Parameter(name="beta_r", expression="5"))
        self.add_parameter(Parameter(name="delta_ma", expression="10"))
        self.add_parameter(Parameter(name="delta_mr", expression="0.5"))
        self.add_parameter(Parameter(name="delta_a", expression="1"))
        self.add_parameter(Parameter(name="delta_r", expression="0.2"))
        self.add_parameter(Parameter(name="gamma_a", expression="1"))
        self.add_parameter(Parameter(name="gamma_r", expression="1"))
        self.add_parameter(Parameter(name="gamma_c", expression="2"))
        self.add_parameter(Parameter(name="theta_a", expression="50"))
        self.add_parameter(Parameter(name="theta_r", expression="100"))

        # Variables
        self.add_species(Species(name="Da", initial_value=1, mode="discrete"))
        self.add_species(Species(name="Da_prime", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Ma", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Dr", initial_value=1, mode="discrete"))
        self.add_species(Species(name="Dr_prime", initial_value=0, mode="discrete"))
        self.add_species(Species(name="Mr", initial_value=0, mode="discrete"))
        self.add_species(Species(name="C", initial_value=10, mode="discrete"))
        self.add_species(Species(name="A", initial_value=10, mode="discrete"))
        self.add_species(Species(name="R", initial_value=10, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="r1", reactants={'Da_prime': 1}, products={'Da': 1}, rate=self.listOfParameters["theta_a"]))
        self.add_reaction(Reaction(name="r2", reactants={'Da': 1, 'A': 1}, products={'Da_prime': 1}, rate=self.listOfParameters["gamma_a"]))
        self.add_reaction(Reaction(name="r3", reactants={'Dr_prime': 1}, products={'Dr': 1}, rate=self.listOfParameters["theta_r"]))
        self.add_reaction(Reaction(name="r4", reactants={'Dr': 1, 'A': 1}, products={'Dr_prime': 1}, rate=self.listOfParameters["gamma_r"]))
        self.add_reaction(Reaction(name="r5", reactants={'Da_prime': 1}, products={'Da_prime': 1, 'Ma': 1}, rate=self.listOfParameters["alpha_a_prime"]))
        self.add_reaction(Reaction(name="r6", reactants={'Da': 1}, products={'Da': 1, 'Ma': 1}, rate=self.listOfParameters["alpha_a"]))
        self.add_reaction(Reaction(name="r7", reactants={'Ma': 1}, products={}, rate=self.listOfParameters["delta_ma"]))
        self.add_reaction(Reaction(name="r8", reactants={'Ma': 1}, products={'A': 1, 'Ma': 1}, rate=self.listOfParameters["beta_a"]))
        self.add_reaction(Reaction(name="r9", reactants={'Da_prime': 1}, products={'Da_prime': 1, 'A': 1}, rate=self.listOfParameters["theta_a"]))
        self.add_reaction(Reaction(name="r10", reactants={'Dr_prime': 1}, products={'Dr_prime': 1, 'A': 1}, rate=self.listOfParameters["theta_a"]))
        self.add_reaction(Reaction(name="r11", reactants={'A': 1}, products={}, rate=self.listOfParameters["gamma_c"]))
        self.add_reaction(Reaction(name="r12", reactants={'A': 1, 'R': 1}, products={'C': 1}, rate=self.listOfParameters["gamma_c"]))
        self.add_reaction(Reaction(name="r13", reactants={'Dr_prime': 1}, products={'Dr_prime': 1, 'Mr': 1}, rate=self.listOfParameters["alpha_r_prime"]))
        self.add_reaction(Reaction(name="r14", reactants={'Dr': 1}, products={'Dr': 1, 'Mr': 1}, rate=self.listOfParameters["alpha_r"]))
        self.add_reaction(Reaction(name="r15", reactants={'Mr': 1}, products={}, rate=self.listOfParameters["delta_mr"]))
        self.add_reaction(Reaction(name="r16", reactants={'Mr': 1}, products={'Mr': 1, 'R': 1}, rate=self.listOfParameters["beta_r"]))
        self.add_reaction(Reaction(name="r17", reactants={'R': 1}, products={}, rate=self.listOfParameters["delta_r"]))
        self.add_reaction(Reaction(name="r18", reactants={'C': 1}, products={'R': 1}, rate=self.listOfParameters["delta_a"]))

        # Timespan
        self.timespan(np.arange(0, 200, 1))

In [ ]:
model = VilarOscillator()

# Simulation Parameters

In [ ]:
def configure_simulation():
    solver = SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

# Model Exploration

## Define simulator function (using gillespy2 wrapper)

In [ ]:
from sciope.utilities.gillespy2 import wrapper
settings = configure_simulation()
simulator = wrapper.get_simulator(gillespy_model=model, run_settings=settings, species_of_interest=['Da', 'Da_prime', 'Ma', 'Dr', 'Dr_prime', 'Mr', 'C', 'A', 'R'])
expression_array = wrapper.get_parameter_expression_array(model)

## Start local cluster using dask client

In [ ]:
from dask.distributed import Client

c = Client()

## Define parameter sampler/design and summary statistics

In [ ]:
from sciope.designs import latin_hypercube_sampling
from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH

lhc = latin_hypercube_sampling.LatinHypercube(xmin=expression_array, xmax=expression_array*3)
lhc.generate_array(1000) # creates a LHD of size 1000

# will use default minimal set of features
summary_stats = SummariesTSFRESH()

## Initiate StochMET

In [ ]:
from sciope.stochmet.stochmet import StochMET

met = StochMET(simulator, lhc, summary_stats)

## Run parameter sweep

In [ ]:
met.compute(n_points=500, chunk_size=10)

## Explore the result

In [ ]:
# First lets add some appropiate information about the model and features
met.data.configurations['listOfParameters'] = list(model.listOfParameters.keys())
met.data.configurations['listOfSpecies'] = list(model.listOfSpecies.keys())
met.data.configurations['listOfSummaries'] = met.summaries.features
met.data.configurations['timepoints'] = model.tspan

In [ ]:
# Here we use UMAP for dimension reduction
met.explore(dr_method='umap')

In [ ]:
from sciope.models.label_propagation import LPModel
# here lets use the dimension reduction embedding as input data
data = met.dr_model.embedding_

model_lp = LPModel()
# train using basinhopping
model_lp.train(data, met.data.user_labels, min_=0.01, max_=10, niter=50)

In [ ]:
# just to vislualize the result we will map the label distribution to the user_labels
# (will enable us to see the LP model output when using method 'explore')
user_labels = np.copy(met.data.user_labels)
# takes the label corresponding to index 0
met.data.user_labels = model_lp.model.label_distributions_[:, 0]

In [ ]:
met.explore(dr_method='umap')

In [ ]:
met.data.user_labels = user_labels